In [1]:
# !pip3 install --upgrade tensorflow-gpu
# !pip install sentence-transformers
# Install TF-Hub.
# !pip3 install tensorflow-hub
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import nltk
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import tensorflow as tf
import tensorflow_hub as hub
# !pip3 install --quiet tensorflow-text
# import tensorflow_text as text

In [2]:
train = pd.read_csv("../input/hackoverflow-ds/dataset/train.csv")
test = pd.read_csv('../input/hackoverflow-ds/dataset/test.csv')
sample = pd.read_csv('../input/hackoverflow-ds/dataset/sample_submission.csv')

In [3]:
train

,Sentence_id,New_Sentence,Type
0,GERRES15609,<html> b'Author and/or Review architecture/des...,Responsibility
1,PHERES15784,<html> b'Should be able to develop custom dyna...,Responsibility
2,GERREQ10457,<html> b'Experience in working cross\\u2010fun...,Requirement
3,GERSKL27235,"<html> b'Previous business experience, includi...",Skill
4,HONSSK18415,b'Delivering fast and right the first \\U0001f...,SoftSkill
...,...,...,...
60110,UAERES18030,"b'In this position, you will \\u2705 utilize y...",Responsibility
60111,GERRES3026,"<html> b'In addition, this individual will be ...",Responsibility
60112,INDSSK5492,<br> b'Good problem \\u2705 solving skills.',SoftSkill
60113,PHESSK15092,b'Good Excel knowledge \\u2705 <br> .',SoftSkill


In [4]:
train['Type'].value_counts()

Responsibility    15561
Requirement       14132
SoftSkill          9581
Experience         9248
Skill              6956
Education          4637
Name: Type, dtype: int64

In [5]:
le = LabelEncoder()
train['Type'] = le.fit_transform(train['Type'])

In [6]:
train

,Sentence_id,New_Sentence,Type
0,GERRES15609,<html> b'Author and/or Review architecture/des...,3
1,PHERES15784,<html> b'Should be able to develop custom dyna...,3
2,GERREQ10457,<html> b'Experience in working cross\\u2010fun...,2
3,GERSKL27235,"<html> b'Previous business experience, includi...",4
4,HONSSK18415,b'Delivering fast and right the first \\U0001f...,5
...,...,...,...
60110,UAERES18030,"b'In this position, you will \\u2705 utilize y...",3
60111,GERRES3026,"<html> b'In addition, this individual will be ...",3
60112,INDSSK5492,<br> b'Good problem \\u2705 solving skills.',5
60113,PHESSK15092,b'Good Excel knowledge \\u2705 <br> .',5


In [7]:
train['Type'].value_counts()

3    15561
2    14132
5     9581
1     9248
4     6956
0     4637
Name: Type, dtype: int64

In [8]:
test

,Sentence_id,New_Sentence
0,UAERES16346,<html> b'Collaborate across all of DB&T practi...
1,COGREQ15586,b'Strong \\u2705 technology expertise in Ident...
2,UAEREQ12722,b'Strong knowledge <br> on Service \\U0001f517...
3,COGSKL29155,<html> b'Architect scalable data processing an...
4,PHERES12551,"<html> b'Map client organization, build outsta..."
...,...,...
15024,HONEXP17084,<html> b'A minimum <p> of 3 </p> years of expe...
15025,UAESSK45553,<html> b'Track record of managing multiple pro...
15026,GERSSK14317,"<html> b'In addition, the candidate should hav..."
15027,PHEREQ4980,b'Candidate should <h1> have sufficient </h1> ...


In [9]:
sample

,Sentence_id,Type
0,GERRES15609,Responsibility
1,PHERES15784,Responsibility
2,GERREQ10457,Requirement
3,GERSKL27235,Skill
4,HONSSK18415,SoftSkill


In [10]:
stop_words = list(stopwords.words('english'))

stop_words.extend(['<', '>','html', ',', '&', '.', ':', "'", '/', '\''])
tfidf = TfidfVectorizer(ngram_range = (1,2), stop_words = stop_words, max_features = 500)
t = tfidf.fit_transform(train['New_Sentence'])
u = tfidf.transform(test['New_Sentence'])
tr = pd.DataFrame(t.toarray(),  index = range(0, train.shape[0]), columns = tfidf.get_feature_names_out())
te = pd.DataFrame(u.toarray(),  index = range(0, test.shape[0]), columns = tfidf.get_feature_names_out())

In [11]:
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
# modl = hub.load(module_url)
# print ("module %s loaded" % module_url)

In [12]:
# preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1')
# # Use BERT pre-processing on a batch of raw text inputs.
# tr_embeddings_bert = preprocess(train['New_Sentence']).numpy()
# te_embeddings_bert = preprocess(test['New_Sentence']).numpy()

In [13]:
# from sentence_transformers import SentenceTransformer
# sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')


In [14]:
# tr_sentence_embeddings = sbert_model.encode(train['New_Sentence']).numpy()
# te_sentence_embeddings = sbert_model.encode(test['New_Sentence']).numpy()
# tr_embeddings_bert

In [15]:
# tr_sentence_embeddings = modl(train['New_Sentence']).numpy()
# te_sentence_embeddings = modl(train['New_Sentence']).numpy()

In [16]:
# tr = pd.DataFrame(tr_sentence_embeddings)
# te = pd.DataFrame(te_sentence_embeddings)

In [17]:
# tr

In [18]:
te_fcol = test[['Sentence_id']]
# cols = [i for i in range(0,10)]
# tr = tr[cols]
# te = te[cols]

In [19]:
train.drop(['Sentence_id', 'New_Sentence'], axis = 1, inplace = True)
test.drop(['Sentence_id', 'New_Sentence'], axis = 1, inplace = True)

train = pd.concat([train, tr], axis = 1)
test = pd.concat([test, te], axis = 1)

In [ ]:
X = train.drop(['Type'], axis = 1)
y = train['Type']

skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
cv_score = []
pred_test_full = []
i = 1
for train_index, test_index in skf.split(X, y):
    print("{} of KFold {}".format(i, skf.n_splits))
    xtr,xvl = X.loc[train_index],X.loc[test_index]
    ytr,yvl = y.loc[train_index],y.loc[test_index]
    sc = StandardScaler()
    nc = MinMaxScaler()
    xt = sc.fit_transform(xtr)
    xv = sc.transform(xvl)
    t = sc.transform(test)
    xtr = pd.DataFrame(nc.fit_transform(xt), columns = xtr.columns)
    xvl = pd.DataFrame(nc.transform(xv), columns = xvl.columns)
    te_df = pd.DataFrame(nc.transform(t), columns = test.columns)
#     model = xgb.XGBClassifier()
    model = RandomForestClassifier(n_jobs = -1, random_state = 42, n_estimators = 200)
#     model = SVC(random_state=42)
    model.fit(xtr, ytr)
    score = max(0, 100*accuracy_score(yvl, model.predict(xvl)))
    print("Accuracy = ", score)
    cv_score.append(score)
    i += 1
    pred_test = model.predict(test)
    pred_test_full.append(pred_test)

1 of KFold 10
Accuracy =  69.84364604125084
2 of KFold 10
Accuracy =  69.16167664670658
3 of KFold 10
Accuracy =  69.24484364604126
4 of KFold 10
Accuracy =  70.14304723885563
5 of KFold 10
Accuracy =  70.35928143712576
6 of KFold 10


In [ ]:
print("MEAN CV = ",np.mean(cv_score))
print("Deviation = ", np.std(cv_score))
print("Final local CV = ", np.mean(cv_score) + np.std(cv_score))
# print("Final PUBLIC CV = ", np.mean(cv_score) + np.std(cv_score) + 0.04833)


In [ ]:
pred_test_full

In [ ]:
mean_preds = np.round(np.mean(pred_test_full, axis = 0))
# med_preds = np.median(pred_test_full, axis = 0)
# mean_preds = np.mean(pred_test_full, axis = 0)

In [ ]:
mean_preds

In [ ]:
submn = pd.DataFrame()
submn['Sentence_id'] = te_fcol['Sentence_id']
submn['Type'] = mean_preds

In [ ]:
submn['Type'].value_counts()

In [ ]:
submn['Type'] = submn['Type'].astype('int')
submn['Type'] = le.inverse_transform(submn['Type'])

In [ ]:
submn.to_csv('Final_Submn_Skf_rf_tfidf_500.csv', index = False)